In [1]:
# 实现卷积核
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 卷积操作，互运算
def corr2d(X, K):
    """X ：输入；K：权重矩阵，包括偏置，也叫作卷积核"""
    h, w = K.shape  # 提取卷积核的长，宽，这是一个h * w的卷积核。我们可以发现，卷积核不一定非要是正方形
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i : i + h, j : j + w] * K).sum()  # 切片后矩阵按元素相乘，后全部相加
    return Y


In [3]:
X = torch.tensor([
    [0.0, 1.0, 2.0],
    [3.0, 4.0, 5.0],
    [6.0, 7.0, 8.0]
])
K = torch.tensor([
    [0.0, 1.0],
    [2.0, 3.0]
])
print(corr2d(X=X, K=K))

tensor([[19., 25.],
        [37., 43.]])


In [4]:
# 卷积层
# 注意卷积层是一个类
# 与原文不同，我们把卷积函数放到卷积层这个类中
class Conv2D(nn.Module):
    """卷积层只有一个初始化参数：卷积核大小"""
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))  # 卷积核随机初始化
        self.bias = nn.Parameter(torch.zeros(1))  # 偏置初始化为0
    
    def corr2d(X, kernel):
        high, wide = kernel.shape
        Y = torch.zeros((X.shape[0] - high + 1, X.shape[1] - wide + 1))
        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                Y[i, j] = (X[i: i + high, j : j + wide] * kernel).sum()
        return Y

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
# 应用 ：图像中目标的边缘检测
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
# 构造卷积核：1 * 2的
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

In [10]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

# 学习卷积核

In [11]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))  # batch_size，通道数，高度，宽度
Y = Y.reshape((1, 1, 6, 7))  
lr = 3e-2

epoch_num = 10

for i in range(epoch_num):
    Y_hat = conv2d(X)
    loss = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    loss.sum().backward()  # .sum() 操作主要用来降维，降到标量，方便反向传播，而sum操作带来的梯度是1，不影响结果
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    print(f'epoch {i + 1}, loss: {loss.sum():.3f}')

epoch 1, loss: 27.861
epoch 2, loss: 11.414
epoch 3, loss: 4.676
epoch 4, loss: 1.916
epoch 5, loss: 0.785
epoch 6, loss: 0.322
epoch 7, loss: 0.132
epoch 8, loss: 0.054
epoch 9, loss: 0.022
epoch 10, loss: 0.009


In [12]:
conv2d.weight.data.reshape(1, 2)

tensor([[ 0.9813, -0.9836]])

# 数学上的卷积和DL中的卷积有什么不同
## 数学上
两个函数$f$ 和 $g$ 之间的卷积被定义为：此处仅考虑离散
$$
(f*g)(x) = \sum_a f(a)g(x - a)
$$
二维：
$$
(f * g)(i, j) = \sum_a\sum_b f(a, b) g(i - a, j - b)
$$
但机器学习中的互相关运算：
## 机器学习
$$
H[i, j] = \sum_a \sum_b V[a, b]X[i + a, j + b]
$$

## 差异
可以发现，两者唯一的差别是“数学中是i - a, j - b, 机器学习中是 i + a, j + b
可以发现，两者的数学形式基本一致，只是运算顺序有区别，但机器学习是一个学习过程，对于同一个数据集，用卷积和互相关得到的核函数是不一样的，但他们都能保证收敛，因此就无所谓了。
